In [ ]:
## FSU MBML seminar
## Dr Eric Dolores

import matplotlib.pyplot as plt
import numpy as np
import torch
import pyro
import pyro.distributions as dist
import pyro.infer
import pyro.optim
import torch.distributions.constraints as constraints


def scale(Data):
    Jskill = pyro.sample('JS', dist.Normal(120.,40.))
    Jperf = pyro.sample('JP', dist.Normal(Jskill, 5.))
    Fskill = pyro.sample('FS', dist.Normal(100.,5.)) 
    Fperf = pyro.sample('FP', dist.Normal(Fskill, 5.))
    Jwin = torch.tensor([0.0])
    if Fperf<Jperf:
        Jwin = torch.tensor([1.0])
    with pyro.plate("plate", len(Data)):
        a= pyro.sample('JW', dist.Bernoulli(Jwin), obs=Data)

def guide(Data):
    J_m = pyro.param('J_m', torch.tensor(120.0))
    J_s = pyro.param('J_s', torch.tensor(40.0), constraint = constraints.positive)
    F_m = pyro.param('F_m', torch.tensor(100.0))
    F_s = pyro.param('F_s', torch.tensor(5.0), constraint=constraints.positive)
    Jp_s = pyro.param('JP_s', torch.tensor(5.0), constraint=constraints.positive)
    Fp_s = pyro.param('FP_s', torch.tensor(5.0),constraint=constraints.positive)
    Jskill = pyro.sample('JS', dist.Normal(J_m,J_s))
    Jperf = pyro.sample('JP', dist.Normal(Jskill, Jp_s))
    Fskill = pyro.sample('FS', dist.Normal(F_m,F_s))
    Fperf = pyro.sample('FP', dist.Normal(Fskill,Fp_s))


Data =  torch.tensor([1.0])

pyro.clear_param_store()
svi = pyro.infer.SVI(model=scale,
                     guide=guide,
                     optim=pyro.optim.SGD({"lr": 0.001, "momentum":0.1}),
                     loss=pyro.infer.Trace_ELBO())


losses, a,b  = [], [], []
num_steps = 2500
for t in range(num_steps):
    losses.append(svi.step(Data))
    a.append(pyro.param("J_s").item())
    b.append(pyro.param("J_m").item())

plt.plot(losses)
plt.title("ELBO")
plt.xlabel("step")
plt.ylabel("loss");
plt.savefig('pt.png')
print('JS = ',pyro.param("J_s").item())
print('JP = ', pyro.param("J_m").item())

